In [1]:
import pandas as pd
import numpy as np
from nltk import jaccard_distance

In [2]:
df=pd.read_csv("/home/shahules786/AI-Matrimony-master/cleaned_data.csv")
df=df.copy()

In [3]:
df.head()

,unique_no,sex,age,body_type,diet,drinks,drugs,education,ethnicity,height,...,sign,smokes,speaks,status,question1,question2,question3,question4,question5,weight
0,0,m,22,fat,anything,socially,never,working on college/university,asian white,75,...,gemini,sometimes,english,single,3,9,4,8,8,56
1,1,m,35,average,other,often,sometimes,working on space camp,white,70,...,cancer,no,english,single,2,9,1,10,9,96
2,2,m,38,thin,anything,socially,never,graduated from masters program,white,68,...,pisces,no,"english,",available,7,3,1,6,5,92
3,3,m,23,thin,vegetarian,socially,never,working on college/university,white,71,...,pisces,no,"english,",single,7,9,3,2,4,93
4,4,m,29,athletic,anything,socially,never,graduated from college/university,asian black other,66,...,aquarius,no,english,single,10,10,9,1,2,78


In [4]:
df_men=df[df['sex']=='m']
df_female=df[df['sex']=='f']
print("The number of male tuples",df_men.shape[0])
print("The number of female tuples",df_female.shape[0])

The number of male tuples 5944
The number of female tuples 4055


## Similarity functions


### Content based similarity functions

**Points to note:**
- Both these functions uses the concept of jaccard similarity to calculate similarity b/w users.
- Both takes single argument,ie..the unique user id of the male user to whom which all the female users will be 
    compared and similarity computed.
- The output is a dataframe that contains unique id of female user and her similarity score with male user.

### Similarity function one

In [5]:
def content_similarity_one(unique):
    
    
    
    columns=['age','height','weight'] ##These are the attributes considered 
    
    va=df_men[df['unique_no']==unique][['age','height','weight']].values[0]
    qa=np.zeros(df_female.shape)
    all_val=[]
    
    for col,v in zip(columns,va):
        max_val = np.max(df[col])
        qa = df_female[col].apply(lambda x : (max_val - np.abs(v-x)) / max_val)
        all_val.append(qa.to_list())
    
    
    similarity = list(zip(all_val[0],all_val[1],all_val[2])) 
    similarity = list (map ( sum, similarity))
    similarity = np.divide(similarity,len(columns))
    
    dataframe=pd.DataFrame({'unique_no':df_female['unique_no'].values,'similarity':similarity}) #creating dataframe
    dataframe.sort_values(by='similarity',ascending=False,inplace=True)
    
    
    return  dataframe[:10]

In [6]:
content_similarity_one(4)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


,unique_no,similarity
2899,7134,1.000000
2993,7415,0.996633
78,202,0.996633
2838,7007,0.996633
2389,5871,0.993939
3236,8103,0.993939
3189,8024,0.993939
765,2036,0.993939
764,2032,0.993939
3805,9459,0.993939


### Similarity function two

In [7]:

def content_similarity_two(unique):
    
    cols=['body_type','diet','drugs','religion','sign','smokes'] ##columns considered
    
    A=set(df_men.iloc[4][cols].to_list())
    qa = df_female[cols].apply(lambda x : 1 - jaccard_distance(A,set(x.to_list())),axis=1)
    
    data_frame= pd.DataFrame({'similarity':qa.values,'unique_no':df_female['unique_no'].values})
    data_frame.sort_values(by='similarity',inplace=True,ascending=False)
    return data_frame
    
    

In [8]:
content_similarity_two(4)[:10]

,similarity,unique_no
851,1.0,2282
1143,1.0,2913
93,1.0,227
3996,1.0,9829
4005,1.0,9849
2432,1.0,5919
3480,1.0,8746
348,1.0,885
1733,1.0,4178
2218,1.0,5364


## Similarity function three

We are creating a new dataframe df2 to store the message sent_id and received_ids.
- We assume that the unique ids 0-500 belong to male users and 500 to 1000 belong to females.
- we are generating values to implement attractiveness similarity function.

In [12]:
df2=pd.DataFrame()

In [13]:
df2['unique_no']=np.arange(0,500)

In [14]:
sent=[]
for idx in range(len(df2)):
    list_sent=np.random.randint(500,1000,np.random.randint(10))
    sent.append(list_sent)
    x=list(map(set,sent))
df2['sent_ids']=x

In [15]:
df2.head()

,unique_no,sent_ids
0,0,"{531, 901, 655}"
1,1,"{645, 646, 752, 697, 953, 604}"
2,2,{921}
3,3,"{649, 858, 723}"
4,4,"{705, 962, 772, 710, 966, 938, 746, 817, 913}"


Data description
- for example, user with unique id 0 has tried contacting female users with ids {531, 901, 655}.

### Interest similarity

In [36]:
def interst_similarity(unique_id):
    
    A = df2.loc[unique_id]['sent_ids']
    
    scores = df2['sent_ids'].apply(lambda x : 1- jaccard_distance(A,x))
    similarity=pd.DataFrame({'unique_id':df2.unique_no,'scores':scores})
    similar_users=similarity.sort_values(by='scores',ascending=False)[:10]['unique_id'].to_list()
  
    similar_users.remove(unique_id)
    
    return similar_users 
    
    
    
    
    
    
    
    
    
    
    

- We will pass the unique_id of user for whom which we want to find users with similar interests.

In [39]:
interst_similarity(1)

[171, 302, 464, 260, 359, 5, 153, 468, 471]

Successfully return top 10 users with similar interests as passed user.